# Iterované vězňovo dilema

In [ ]:
import random

import numpy as np

from deap import base, creator, tools, algorithms

## Strategie odpovědí
obecně jde o funkci tvaru *zrada(moje_historie_tahu, druheho_historie_tahu)* pro seznam minulých rozhodnutí vrátí, zda protihráče v následujícím tahu zradíme (1) nebo ne (0)

- první parametr je seznam historie našich tahů
- druhý parametr je seznam historie tahů protihráče


In [ ]:

# vždy kooperuje
def always_cooperate(myhistory, otherhistory):
    return 0


# náhodná odpověď
def random_answer(myhistory, otherhistory):
    p = random.random()
    if p < 0.5:
        return 1

    return 0





## Pomocné funkce

In [ ]:
def rozdej_skore(tah1, tah2):
    # 1 = zradi, 0 = nezradi

    skores = (0, 0)

    if (tah1 == 1) and (tah2 == 1):
        skores = (2, 2)

    if (tah1 == 1) and (tah2 == 0):
        skores = (0, 3)

    if (tah1 == 0) and (tah2 == 1):
        skores = (3, 0)

    if (tah1 == 0) and (tah2 == 0):
        skores = (1, 1)

    return skores


def play(f1, f2, stepsnum):

    skore1 = 0
    skore2 = 0

    historie1 = []
    historie2 = []

    for i in range(stepsnum):
        tah1 = f1(historie1, historie2)
        tah2 = f2(historie2, historie1)

        s1, s2 = rozdej_skore(tah1, tah2)
        skore1 += s1
        skore2 += s2

        historie1.append(tah1)
        historie2.append(tah2)

    return skore1, skore2



## Evoluce strategie

In [ ]:
# místo pro vlastní evoluci strategie

## Moje funkce

In [ ]:
# vlastní funkce zrada(moje_historie_tahu, druheho_historie_tahu)
# zadrátovaná dle

## Turnaj

In [ ]:


# seznam funkci o testování
ucastnici = [always_cooperate, random_answer]

# funkce se mohou v seznamu i opakovat
#ucastnici = [always_cooperate, always_cooperate, random_answer, random_answer, random_answer]



STEPSNUM = 20


l = len(ucastnici)
skores = [0 for i in range(l)]

print("=========================================")
print("Turnaj")
print("hra délky:", STEPSNUM)
print("-----------------------------------------")


for i in range(l):
    for j in range(i+1, l):
        f1 = ucastnici[i]
        f2 = ucastnici[j]
        skore1, skore2 = play(f1, f2, STEPSNUM)
        print(f1.__name__, "x", f2.__name__, " ", skore1, ":", skore2)
        skores[i] += skore1
        skores[j] += skore2


print("=========================================")
print("= Výsledné pořadí")
print("-----------------------------------------")


# setrideni indexu vysledku
index = sorted(range(l), key=lambda k: skores[k])

poradi = 1
for i in index:
    f = ucastnici[i]
    print(poradi, ".", f.__name__, ":", skores[i])
    poradi += 1

